## **Apache Beam Basics**

Description: This notebook will teach you the basics of the Apache Beam Programming Model:

- Apache Beam Basics: Pipeline, PCollection, Transforms & PTransforms.
   - GroupByKey
   - CoGroupByKey
   - Flatten
   - Partition
- Introducing complexity: DoFn.
   - DoFn vs Map
   - DoFn LifeCycle
   - DoFn Stateful Processing
- Advanced: Streaming.
   - Fixed Windows
   - Sliding Windows
   - PubSub to Bigquery


EDEM. Master Big Data & Cloud 2025/2026<br>
Professor: Javi Briones

In [ ]:
!pip install "apache-beam[gcp]" --quiet

In [ ]:
# Import libraries
import apache_beam as beam
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner
import apache_beam.runners.interactive.interactive_beam as ib
import logging

In [ ]:
# Set logs
logging.basicConfig(level=logging.INFO, format="edem_apache_beam-%(message)s")

# Set Apache Beam log level to WARNING
logging.getLogger('apache_beam').setLevel(logging.WARNING)

## Apache Beam Basics

Apache Beam is a unified programming model for parallel data processing that provides a rich set of transformations to efficiently manipulate and process both batch and streaming data across multiple execution environments

### **Apache Beam Core Concepts** 

##### **Pipeline**
A **Pipeline** represents the entire workflow for processing data in Apache Beam. It defines the sequence of transformations applied to input data to produce outputs.

##### **PCollection**
A **PCollection** is an immutable, distributed dataset that serves as the input and output for transformations in a pipeline. It can handle bounded (batch) or unbounded (streaming) data.

##### **Transformations**
A **Transformation** is an operation applied to a PCollection to produce one or more output PCollections. Common transformations include Map, FlatMap, GroupByKey, and Combine.

##### **PTransform**
A **PTransform** is a reusable and composable abstraction that encapsulates one or more transformations. It simplifies pipelines by modularizing complex workflows.


In [ ]:
"""
Exercise 01: What is a PCollection?

PCollection: A distributed dataset in Apache Beam
"""

#  Create and explore a simple PCollection

with beam.Pipeline(InteractiveRunner()) as pipeline:
    
    # Create a PCollection
    numbers = pipeline | "Create Numbers" >> beam.Create([1, 2, 3, 4, 5])

    # Log output
    numbers | "Log Elements" >> beam.Map(logging.info)

In [ ]:
"""
Exercise 02: Declaring a Pipeline

Pipeline: Defines the workflow of data transformations.
"""

# Interactive pipeline
with beam.Pipeline(InteractiveRunner()) as pipeline:
    
    # Create a PCollection and perform a transformation
    squared_numbers = (
        pipeline
        | "Create Numbers" >> beam.Create([1, 2, 3, 4, 5])
        | "Square Numbers" >> beam.Map(lambda x: x * x)
    )

    # Log output
    squared_numbers | "Log Squared Numbers" >> beam.Map(logging.info)

In [ ]:
"""
Exercise 03: Transformations

Transforms one or more input PCollections into one or more output PCollections.
"""

# Interactive pipeline
with beam.Pipeline(InteractiveRunner()) as pipeline:

    (
        pipeline
            # Input
            | "Input PCollection: Read Text From File" >> beam.io.ReadFromText(
                './00_DocAux/input_text.txt')
            # Transformations
            | "FlatMap" >> beam.FlatMap(lambda z: z.split())
            | "Map" >> beam.Map(lambda x: (x,1))
            | "Combine" >> beam.CombinePerKey(sum)
            # Output
            | "Output PCollection: Print" >> beam.Map(logging.info)
    )

In [ ]:
"""
Exercise 04: Reuse Transformations (PTransforms)

PTransform: A collection of transformations packaged as a reusable unit for complex workflows.
"""

class MyCustomTransform(beam.PTransform):
    
    def expand(self, pcoll):
        return (
            pcoll
            | "FlatMap" >> beam.FlatMap(lambda z: z.split())
            | "Map" >> beam.Map(lambda x: (x,1))
            | "Combine" >> beam.CombinePerKey(sum)
        )

with beam.Pipeline(InteractiveRunner()) as pipeline:

    (
        pipeline
            | "Input PCollection: Read Text From File" >> beam.io.ReadFromText('./00_DocAux/input_text.txt')
            | "Apply Custom Transform" >> MyCustomTransform()
            | "Output PCollection: Print" >> beam.Map(logging.info)
    )

#### **Apache Beam Transformations** 

**A. GroupByKey**

It groups the elements of a data bundle by a common key, producing a collection where each key is unique and associated with a list of values that share that key.


In [ ]:
""" GroupByKey """

with beam.Pipeline(InteractiveRunner()) as p:

    data = (p | "PCollection" >> beam.Create([('Spain', 'Valencia'), ('Spain','Barcelona'), ('France', 'Paris')]))

    (data 
        | "Combined" >> beam.GroupByKey()
        | "Print" >> beam.Map(logging.info))

**B. CoGroupByKey** 

It merges two PCollections by key, producing pairs where each key is associated with lists of elements from both input collections. This is useful for operations involving data from two different sources.


In [ ]:
with beam.Pipeline(InteractiveRunner()) as p:

    p1 = p | "PCollection 01" >> beam.Create([('Spain', 'Valencia'), ('Spain','Barcelona'), ('France', 'Paris')])
    p2 = p | "PCollection 02" >> beam.Create([('Spain', 'Madrid'), ('Spain','Alicante'), ('France', 'Lyon')])

    data = ((p1,p2) | beam.CoGroupByKey())

    data | "Print" >> beam.Map(logging.info)

**C. Flatten**  

It merges multiple PCollections into a single PCollection, combining their elements into one flat structure.

In [ ]:
with beam.Pipeline(InteractiveRunner()) as p:

    p1 = p | "PCollection 01" >> beam.Create(['New York', 'Los Angeles', 'Miami', 'Chicago'])
    p2 = p | "Pcollection 02" >> beam.Create(['Madrid', 'Barcelona', 'Valencia', 'Malaga'])
    p3 = p | "Pcollection 03" >> beam.Create(['London','Manchester', 'Liverpool'])

    merged = ((p1,p2,p3)| beam.Flatten())

    merged | beam.Map(logging.info)

**D. Partition**  

It splits a PCollection into multiple partitions based on defined criteria, enabling parallel and distributed processing of subsets of the data.


In [ ]:
countries = ['Spain', 'USA', 'Switzerland']

def choose_partition(city, num_partitions):
    
    usa = {'New York', 'Los Angeles', 'Miami', 'Chicago'}
    eu  = {'Madrid', 'Barcelona', 'Valencia', 'Malaga'}
    uk  = {'London', 'Manchester', 'Liverpool'}

    if city in usa:
        return 0  
    if city in eu:
        return 1  
    if city in uk:
        return 2 
    return 3 

with beam.Pipeline(InteractiveRunner()) as p:

        usa,eu,uk,others = (
                p 
                | "PCollection" >> beam.Create([
                        'New York', 'Los Angeles', 'Miami', 'Chicago',
                        'Madrid', 'Barcelona', 'Valencia', 'Malaga',
                        'London', 'Manchester', 'Liverpool','Tokyo' 
                ])
                | "partition" >> beam.Partition(choose_partition, 4)
        )

        eu |  beam.Map(logging.info)

## Introducing complexity

In [ ]:
"""
Exercise 05: This exercise shows how to achieve the same transformation
    using both Map and ParDo with DoFn.

Use Map for simple transformations where the logic is inline and does not require lifecycle methods.
Use ParDo with DoFn for more complex operations requiring setup, cleanup, or state management.
"""

In [ ]:
""" Map """
with beam.Pipeline(InteractiveRunner()) as pipeline:

    (
        pipeline
            | "Input PCollection: Read Text From File" >> beam.io.ReadFromText('./00_DocAux/input_text.txt')
            | "FlatMap" >> beam.FlatMap(lambda z: z.split())
            | "Map" >> beam.Map(lambda x: x.upper())
            | "Output PCollection: Print" >> beam.Map(logging.info)
    )

In [ ]:
"""ParDo with DoFn"""

class UpperCaseDoFn(beam.DoFn):

    def setup(self):
        logging.info("Loading model...")
        self.model = lambda x: x.upper()

    def process(self, element):
        yield self.model(element)

    def teardown(self):
        logging.info("Releasing model resources.")

with beam.Pipeline(InteractiveRunner()) as pipeline:

    (
        pipeline
            | "Input PCollection: Read Text From File" >> beam.io.ReadFromText('./00_DocAux/input_text.txt')
            | "FlatMap" >> beam.FlatMap(lambda z: z.split())
            | "DoFn" >> beam.ParDo(UpperCaseDoFn())
            | "Output PCollection: Print" >> beam.Map(logging.info)
    )


### **Exploring the DoFn LifeCycle**

The Lifecycle of a **DoFn** in Apache Beam refers to the stages that an instance of the `DoFn` class goes through from initialization to completion within the context of a *ParDo* transformation. Below are the key phases of the DoFn life cycle:

1. **Setup (Initialization):**  
   - **Goal:** This phase occurs once for each instance of `DoFn` before processing begins.  
   - **Process:** Initialization tasks, such as allocating resources, establishing connections, or loading models, are performed in the `setup(self)` method.

2. **Processing Elements (Process):**  
   - **Goal:** This phase executes the main logic for each input element in the data bundle.  
   - **Process:** The core processing logic is implemented in the `process(self, element)` method. This method is called for each element in the input `PCollection` and defines how the data is transformed.

3. **Start Bundle and Finish Bundle:**  
   - **Goal:** These phases occur once before (`start_bundle(self)`) and after (`finish_bundle(self)`) processing all elements in a bundle. A bundle represents a chunk of data processed in parallel.  
   - **Process:** Bundle-specific setup and cleanup tasks, such as initializing or finalizing temporary states or resources used within the bundle, are performed here.

4. **Teardown (Finalization):**  
   - **Goal:** This phase occurs once after all elements have been processed.  
   - **Process:** The `teardown(self)` method handles final resource cleanup, such as closing database connections, releasing memory, or terminating external processes.

---

The lifecycle of a **DoFn** provides a structured framework for managing resources and processing logic in *ParDo* transformations. Each instance is created, initialized, processes elements, and performs cleanup in a controlled and efficient manner. This ensures proper resource management and facilitates robust, scalable data processing pipelines.


In [ ]:
"""
Exercise: Understanding the lifecycle methods (setup, start_bundle, process, finish_bundle, and teardown)
 and how they are invoked during pipeline execution.

Use ParDo with DoFn for tasks requiring state, lifecycle, or multiple outputs.
"""

class LifecycleDoFn(beam.DoFn):

    def setup(self):
        logging.info("Setting up resources.")

    def start_bundle(self):
        logging.info("Starting a bundle.")

    def process(self, element):
        logging.info(f"Processing element: {element}")
        yield element.upper()

    def finish_bundle(self):
        logging.info("Finishing a bundle.")

    def teardown(self):
        logging.info("Tearing down resources.")

with beam.Pipeline(InteractiveRunner()) as pipeline:

    (
        pipeline
            | "Input PCollection: Read Text From File" >> beam.io.ReadFromText('./00_DocAux/input_text.txt')
            | "FlatMap" >> beam.FlatMap(lambda z: z.split())
            | "Apply Lifecycle DoFn" >> beam.ParDo(LifecycleDoFn())
            | "Output PCollection: Print" >> beam.Map(logging.info)
    )



### **DoFn: Stateful Processing**

- Allows operations that depend on previous or cumulative data.
- Reduces the need to rely on external systems, such as databases or caching systems.
- Efficient, as the state is stored locally for each key and window.


In [ ]:
from apache_beam.transforms.userstate import CombiningValueStateSpec

"""
Exercise: The state is persisted across multiple calls to process()
"""

class StatefulDoFn(beam.DoFn):
    
    # Define a state spec to maintain counts
    state_spec = CombiningValueStateSpec("count", sum)

    def process(self, element, state=beam.DoFn.StateParam(state_spec)):
        k,v = element
        state.add(1)
        yield f"Key: {k}, Count: {state.read()}"
    
# Interactive pipeline
with beam.Pipeline(InteractiveRunner()) as pipeline:

    (
        pipeline
            | "Input PCollection: Read Text From File" >> beam.io.ReadFromText('./00_DocAux/input_text.txt')
            | "FlatMap" >> beam.FlatMap(lambda z: z.split())
            | "Map" >> beam.Map(lambda x: (x.lower(),1))
            | "Count Events Per Key" >> beam.ParDo(StatefulDoFn())
            | "Output PCollection: Print" >> beam.Map(logging.info)
    )

### Beam ML

- The **RunInference** transform enables the execution of machine learning inference directly within an Apache Beam pipeline by applying a preconfigured model to incoming data elements

- **MLTransform** is used within a pipeline to prepare data for machine learning model training. It encapsulates multiple data preprocessing operations into a single abstraction, enabling the application of diverse transformation workflows through a unified interfac

In [ ]:
!pip install torch==2.6.0 --quiet
!pip install tensorflow==2.18.0 --quiet
!pip install transformers==4.44.2 --quiet

In [ ]:
from typing import Dict
from typing import Iterable
from typing import Tuple

import tensorflow as tf
import torch
from transformers import AutoTokenizer
from transformers import TFAutoModelForMaskedLM

import apache_beam as beam
from apache_beam.ml.inference.base import KeyedModelHandler
from apache_beam.ml.inference.base import PredictionResult
from apache_beam.ml.inference.base import RunInference
from apache_beam.ml.inference.huggingface_inference import HuggingFacePipelineModelHandler
from apache_beam.ml.inference.huggingface_inference import HuggingFaceModelHandlerKeyedTensor
from apache_beam.ml.inference.huggingface_inference import HuggingFaceModelHandlerTensor
from apache_beam.ml.inference.huggingface_inference import PipelineTask

In [ ]:
model_handler = HuggingFacePipelineModelHandler(
    task=PipelineTask.Translation_XX_to_YY,
    model = "google/flan-t5-small",
    load_pipeline_args={'framework': 'pt', "device": -1},
    inference_args={'max_new_tokens': 200}
)

class FormatOutput(beam.DoFn):
    def process(self, element: PredictionResult[Dict]) -> Iterable[Tuple[str, str]]:
        
        translated_text = element.inference[0]['translation_text']

        logging.info(f"Translated Text: {translated_text}")
        
        yield translated_text

with beam.Pipeline() as pipeline:

    (
        pipeline
            | "Input PCollection: Read Text From File" >> beam.io.ReadFromText('./00_DocAux/input_text.txt')
            | "RunInference" >> RunInference(model_handler)
            | "Print" >> beam.ParDo(FormatOutput())
    )


### **Streaming** 

#### GCP Setup

- PubSub Topics

```
gcloud pubsub topics create <TOPIC_NAME>
```

- PubSub Subscriptions

```
gcloud pubsub subscriptions create <SUBSCRIPTION_NAME> --topic=<TOPIC_NAME>
```

- Google Cloud Storage Bucket  

```
gcloud storage mb gs://<BUCKET_NAME> --location=<REGION_ID>
```


In [ ]:
# Variables
project_id = ""
subscription_name = ""
bq_dataset = ""
bq_table = ""
bucket_name = ""

**Window in Apache Beam**
  
A window in Apache Beam defines a time frame for **organizing and grouping data elements** during processing, enabling time-based operations.

---

**Types of Windows:**

- **Fixed Window:** 

  Divides elements into fixed time intervals, segmenting the PCollection into evenly spaced, time-based windows.

- **Sliding Window:**  

  Creates overlapping windows with a specified size and stride, enabling continuous analysis of data over time.

- **Session Window:**  

  Groups elements based on **contiguous temporal activity**, where windows are dynamically defined by an **inactivity gap** between events, making it ideal for capturing logical sessions in streaming data.


**A. FixedWindows**

In [ ]:
from apache_beam.transforms.window import FixedWindows

class getWindowDoFn(beam.DoFn):
    def process(self, element, window=beam.DoFn.WindowParam):
        yield (str(window), element)

with beam.Pipeline(InteractiveRunner()) as p:
    events = [
        beam.window.TimestampedValue("event1", 60),   
        beam.window.TimestampedValue("event2", 180),  
        beam.window.TimestampedValue("event3", 360), 
    ]

    (
        p
        | "Create PCollection" >> beam.Create(events)
        | "Sliding Window" >> beam.WindowInto(FixedWindows(size=300))
        | "k,v" >> beam.Map(lambda x: ("key", x))
        | "Combine" >> beam.GroupByKey()
        | "Print" >> beam.Map(print)
    )


**B. SlidingWindows**

In [ ]:
from apache_beam.transforms.window import SlidingWindows

class getWindowDoFn(beam.DoFn):
    def process(self, element, window=beam.DoFn.WindowParam):
        if window.start > 0:
            yield (str(window), element)

with beam.Pipeline(InteractiveRunner()) as p:
    events = [
        beam.window.TimestampedValue("event1", 1),   
        beam.window.TimestampedValue("event2", 7),  
        beam.window.TimestampedValue("event3", 9), 
    ]

    (
        p
        | "Create PCollection" >> beam.Create(events)
        | "Sliding Window" >> beam.WindowInto(SlidingWindows(size=10, period=2))
        | "k,v" >> beam.ParDo(getWindowDoFn())
        | "Combine" >> beam.GroupByKey()
        | "Print" >> beam.Map(print)
    )


**C. SessionWindows**

In [ ]:
from apache_beam.transforms.window import Sessions

with beam.Pipeline(InteractiveRunner()) as p:
    events = [
        beam.window.TimestampedValue("event1", 0),   
        beam.window.TimestampedValue("event2", 5),  
        beam.window.TimestampedValue("event3", 25),
        beam.window.TimestampedValue("event4", 27), 
    ]

    (
        p
        | "Create PCollection" >> beam.Create(events)
        | "Sliding Window" >> beam.WindowInto(Sessions(gap_size=7))
        | "k,v" >> beam.Map(lambda x: ("key", x))
        | "Combine" >> beam.GroupByKey()
        | "Print" >> beam.Map(print)
    )


**D. Triggers, Watermark & Allowed Lateness**

In [ ]:
import apache_beam as beam
from apache_beam.testing.test_stream import TestStream
from apache_beam.transforms.window import FixedWindows
from apache_beam.transforms.trigger import AfterWatermark, AfterProcessingTime
from apache_beam.transforms.trigger import AccumulationMode
from apache_beam.options.pipeline_options import PipelineOptions

opts = PipelineOptions(flags=["--allow_unsafe_triggers"],streaming=True, runner="DirectRunner")

with beam.Pipeline(options=opts) as p:
    events = (
        TestStream()
        .add_elements([
            beam.window.TimestampedValue("event1", 0),
            beam.window.TimestampedValue("event2", 2),
        ])
        .advance_processing_time(5)
        .add_elements([
            beam.window.TimestampedValue("event3", 4),
        ])
        .advance_watermark_to(10)
        .add_elements([
            beam.window.TimestampedValue("late_event", 1),
        ])
        .advance_watermark_to_infinity()
    )

    (
        p
        | "Stream" >> events
        | "Window" >> beam.WindowInto(
            FixedWindows(5),
            trigger=AfterWatermark(early=AfterProcessingTime(3)),
            allowed_lateness=10,
            accumulation_mode=AccumulationMode.ACCUMULATING
        )
        | "k,v" >> beam.Map(lambda x: ("key", x))
        | "Group" >> beam.GroupByKey()
        | "Print" >> beam.Map(print)
    )


### Apache Beam on Google Cloud: Dataflow Pipeline from Pub/Sub to BigQuery

In [ ]:
from apache_beam.options.pipeline_options import PipelineOptions
import apache_beam as beam
import logging
import json
import sys

sys.argv = ['']

def decode_message(msg):

    output = msg.decode('utf-8')
    logging.info("New PubSub Message: %s", output)

    return json.loads(output)

def run():
    with beam.Pipeline(options=PipelineOptions(
        streaming=True,
        job_name = "edem-pubsub-to-bigquery",
        project=project_id,
        runner="DataflowRunner",
        temp_location=f"gs://{bucket_name}/tmp",
        staging_location=f"gs://{bucket_name}/staging",
        region="europe-southwest1"
    )) as p:
        
        (
            p 
            | "ReadFromPubSub" >> beam.io.ReadFromPubSub(subscription=f'projects/{project_id}/subscriptions/{subscription_name}')
            | "decode msg" >> beam.Map(decode_message)
            | "Write to BigQuery" >> beam.io.WriteToBigQuery(
                table = f"{project_id}:{bq_dataset}.{bq_table}", # Required Format: PROJECT_ID:DATASET.TABLE
                schema='name:STRING', # Required Format: field_name:TYPE
                create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED,
                write_disposition=beam.io.BigQueryDisposition.WRITE_APPEND
            )
        )

# Run Process
logging.info("The process started")
run()